In [ ]:
# 상품명 가져오는 코드

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://prod.danawa.com/list/?cate=112758"

# 페이지 요청
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
html = requests.get(url, headers=header)
soup = BeautifulSoup(html.content, "html.parser")

# "prod_main_info" 클래스를 갖는 div 태그들을 찾습니다.
div_tags = soup.find_all("div", class_="prod_main_info")

lst_name = []
for div_tag in div_tags:
    # a 태그 중 name 속성이 "productName"인 요소를 찾습니다.
    title = div_tag.find("a", attrs={"name": "productName"})
    

    if title is not None:
        # a 태그 내용 출력
        # print(title.text.strip())
        lst_name.append(title.text.strip())
# lst_name
index = pd.Index(range(1, len(lst_name) + 1))
df_title = pd.DataFrame(lst_name, index=index)
df_title.columns = ['노트북 이름']
df_title

## [ 노트북 / 운영체제 / 뭐설치됬는지 ] 딱 이거 3개만 데이터프레임 생성

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://prod.danawa.com/list/?cate=112758"

# 페이지 요청
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
html = requests.get(url, headers=header)
soup = BeautifulSoup(html.content, "html.parser")

# div.prod_main_info 요소 추출
prod_main_info_divs = soup.select('div.prod_main_info')

# 결과를 저장할 리스트
result_list = []

# div.prod_main_info 요소를 반복하여 처리
for prod_main_info_div in prod_main_info_divs:
    # div.spec_list 요소 추출
    spec_list_div = prod_main_info_div.find('div', class_='spec_list')

    # div.spec_list가 존재하는 경우에만 작업 수행
    if spec_list_div:
        # a 태그 내용 불러오기
        a_tags = spec_list_div.select('a')

        # 맨 처음 3개의 a 태그 내용을 저장
        contents = [a_tag.text.strip() for a_tag in a_tags[:3]]
        result_list.append(contents)

# 결과 리스트 출력
# print(result_list)
index = pd.Index(range(1, len(result_list) + 1))
df_a3 = pd.DataFrame(result_list, index=index)[:-1]
del df_a3[1]
df_a3.columns = ['노트북유형', '운영체제']
df_a3

# 딕셔너리 형태로 만드는것도 성공함. - 이걸로 쓰자!!

import requests
from bs4 import BeautifulSoup

url = "https://prod.danawa.com/list/?cate=112758"

# 페이지 요청
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}
html = requests.get(url, headers=header)
soup = BeautifulSoup(html.content, "html.parser")

# div.prod_main_info 요소 추출
prod_main_info_divs = soup.select('div.prod_main_info')

# 결과를 저장할 리스트
result_list = []

# div.prod_main_info 요소를 반복하여 처리
for prod_main_info_div in prod_main_info_divs:
    # div.spec_list 요소 추출
    spec_list_div = prod_main_info_div.find('div', class_='spec_list')

    # div.spec_list가 존재하는 경우에만 작업 수행
    if spec_list_div:
        # span.cm_mark 태그 내용 불러오기
        cm_mark_tags = spec_list_div.select('span.cm_mark')

        # span.cm_mark 태그 뒤에 오는 모든 내용 추출
        spec_dict = {}
        for cm_mark_tag in cm_mark_tags:
            # 현재 cm_mark 태그의 다음 sibling 요소들 중 텍스트 추출
            next_texts = []
            next_element = cm_mark_tag.next_sibling
            while next_element and not (next_element.name == 'span' and 'cm_mark' in next_element.get('class', [])):
                if next_element.name == 'a':
                    next_texts.append(next_element.get_text(strip=True))
                elif isinstance(next_element, str) and next_element.strip():
                    next_texts.append(next_element.strip())
                next_element = next_element.next_sibling

            if next_texts:
                spec_dict[cm_mark_tag.get_text(strip=True)] = [next_texts]
                # 데이터프레임을 바로 만들기 위해서 []에 넣어줬음

        if spec_dict:
            result_list.append(spec_dict)



result_df_spec = pd.DataFrame()
for i in range(len(result_list)):
    df_spec = pd.DataFrame.from_dict(result_list[i], orient='index').transpose()
    result_df_spec = pd.concat([result_df_spec, df_spec])

result_df_spec.index = range(1, len(result_df_spec) + 1)
result_df_spec


# 가격 데이터 크롤링하는 코드

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://prod.danawa.com/list/?cate=112758"

# 페이지 요청
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}
html = requests.get(url, headers=header)
soup = BeautifulSoup(html.content, "html.parser")

# 크롤링
prod_main_info_list = soup.find_all("div", class_="prod_main_info")
result_list = []

for prod_main_info in prod_main_info_list:
    prod_pricelist = prod_main_info.find("div", class_="prod_pricelist")
    if prod_pricelist:
        li_list = prod_pricelist.find_all("li")
        sub_list = []  # 내부 리스트
        for li in li_list:
            price_sect = li.find("p", class_="price_sect")
            if price_sect:
                a_tag = price_sect.find("a")
                if a_tag:
                    strong_tag = a_tag.find("strong")
                    if strong_tag:
                        value = strong_tag.text.strip()
                        sub_list.append(value)
        if sub_list:  # 내부 리스트가 비어 있지 않은 경우에만 추가
            result_list.append(sub_list)

result_list = result_list[4:-1]

if len(result_list) > 30:
    del result_list[10]
# print(result_list)
index = pd.Index(range(1, len(result_list) + 1))
df_price = pd.DataFrame([result_list], columns = index)
df_price = df_price.T

df_price.columns = ['가격']
df_price


merged_df = pd.concat([df_title, df_price, df_a3, result_df_spec], axis=1)
merged_df


,노트북 이름,가격,노트북유형,운영체제,화면정보,CPU,램,그래픽,저장장치,네트워크,영상입출력,단자,부가기능,입력장치,파워,주요제원
1,MSI 모던시리즈 모던 14 C7M-R5,"[504,430, 735,000, 880,000]",노트북,미포함(프리도스),"[35.5cm(14인치), /, 1920x1080(FHD), /, 300nit, /]","[AMD, /, 라이젠5(Zen3), / 7530U (2.0Hz) /, 헥사(6)코...","[DDR4(온보드), /, 램 용량, :, 8GB, /, 3200MHz, / 램 교...",[내장그래픽 / Radeon Graphics /],"[M.2(NVMe), /, 512GB, /]",[무선랜: 802.11ax(Wi-Fi 6E) /],"[HDMI, /, 웹캠(HD), /]",[USB-C: 1개 / USB-A: 3개 / MicroSD카드 /],"[USB-PD, / 리프트 힌지 /]","[키보드 라이트, /, ㅗ형 방향키, /]","[배터리, :, 39.3Wh, /, 어댑터, :, 65W, /, 충전단자, :, D...","[두께, :, 19.35mm, /, 무게, :, 1.4kg, / 색상: 블랙]"
2,삼성전자 갤럭시북2 NT550XEZ-A58A 16GB램,"[673,890, 721,900, 771,990, 946,000, 739,000, ...",노트북,미포함(프리도스),"[39.6cm(15.6인치), /, 1920x1080(FHD), /]","[인텔, /, 코어i5-12세대, /, i5-1235U (1.3GHz), /, 10...","[DDR4, /, 램 용량, :, 16GB, / 램 교체: 가능 /]",[내장그래픽 / UHD Graphics /],"[M.2(NVMe), /, 256GB, /, 저장 슬롯, :, 2개, /]","[무선랜:, 802.11ax(Wi-Fi 6), / 유선랜:, 기가비트, /]","[HDMI, /, 웹캠(HD), /]",[USB-C: 2개 / USB-A: 2개 / MicroSD카드 /],"[TPM, /, USB-PD, /]","[ㅗ형 방향키, /, 숫자 키패드(3열), /]","[배터리, :, 43Wh, /, 어댑터, :, 45W, /, 충전단자, :, 타입C...","[두께, :, 18.5mm, /, 무게, :, 1.8kg, / 색상: 실버, 그라파이트]"
3,한성컴퓨터 TFG6397,"[2,290,000]",노트북,미포함(프리도스),"[40.64cm(16인치), /, 2560x1600(WQXGA), /, 500nit...","[인텔, /, 코어i9-13세대, /, i9-13900HX (2.2GHz), / 2...","[DDR5, /, 램 용량, :, 16GB, /, 4800MHz, / 램 교체: 가...","[외장그래픽 /, RTX4070, /, TGP, :, 140W, /, VRAM:8G...","[M.2(NVMe), /, 500GB, /, 저장 슬롯, :, 2개, /]",[무선랜: 802.11ax(Wi-Fi 6E) / 유선랜: 2.5기가비트 /],"[HDMI 2.1, /, 웹캠(HD), /]","[썬더볼트4, :, 1개(USB-C겸용), / USB-A: 3개 / SD카드 /]","[얼굴 인식, /, DP Alt Mode, /]","[RGB 라이트, /, ㅗ형 방향키, /, 숫자 키패드(3열), /]","[배터리, :, 97.5Wh, /, 어댑터, :, 330W, /, 충전단자, :, ...","[두께, :, 31mm, /, 무게, :, 2.6kg, / 쿨링팬: 2개 / 색상:..."
4,MSI GF시리즈 Sword GF76 B13VFK,"[1,499,000, 1,554,460, 1,865,620, 1,678,790, 1...",노트북,미포함(프리도스),"[43.9cm(17.3인치), /, 1920x1080(FHD), /, 250nit,...","[인텔, /, 코어i7-13세대, /, i7-13620H (2.4GHz), /, 1...","[DDR5, /, 램 용량, :, 16GB, / 램 교체: 가능 /]","[외장그래픽 /, RTX4060, /, TGP, :, 105W, /, VRAM:8G...","[M.2(NVMe), /, 512GB, /, 저장 슬롯, :, 2개, /]","[무선랜:, 802.11ax(Wi-Fi 6), / 유선랜:, 기가비트, /]","[HDMI, /, 웹캠(HD), /]",[USB-C: 1개 / USB-A: 3개 /],"[DP Alt Mode, /]","[RGB 라이트, /, ㅗ형 방향키, /, 숫자 키패드(3열), /]","[배터리, :, 53.5Wh, /, 충전단자, :, DC, /]","[두께, :, 25.2mm, /, 무게, :, 2.6kg, / 쿨링팬: 2개 / 색..."
5,한성컴퓨터 Bossmonster GTX63 T9X6Q16,"[1,799,000]",노트북,미포함(프리도스),"[40.64cm(16인치), /, 2560x1600(WQXGA), /, 500nit...","[인텔, /, 코어i9-13세대, /, i9-13900HX (2.2GHz), / 2...","[DDR5, /, 램 용량, :, 16GB, /, 4800MHz, / 램 교체: 가...","[외장그래픽 /, RTX4060, /, TGP, :, 140W, /, VRAM:8G...","[M.2(NVMe), /, 500GB, /, 저장 슬롯, :, 2개, /]","[무선랜: 802.11ax(Wi-Fi 6E) / 유선랜:, 기가비트, /]","[HDMI 2.0, /, mini DP, /, 웹캠(HD), /]","[썬더볼트4, :, 1개(USB-C겸용), / USB-C: 1개 / USB-A: 2...",NaN,"[RGB 라이트, /, ㅗ형 방향키, /, 숫자 키패드(4열), /]","[배터리, :, 73Wh, /, 어댑터, :, 280W, /, 충전단자, :, DC...","[두께, :, 27.8mm, /, 무게, :, 2.5kg, / 쿨링팬: 2개 / 색..."
6,한성컴퓨터 Bossmonster GTX73 N9X6Q24,"[1,788,000]",노트북,미포함(프리도스),"[43.94cm(17.3인치), /, 2560x1440(QHD), /, 300nit...","[인텔, /, 코어i9-13세대, /, i9-13900HX (2.2GHz), / 2...","[DDR5, /, 램 용량, :, 16GB, /, 4800MHz, / 램 교체: 가...","[외장그래픽 /, RTX4060, /, TGP, :, 140W, /, VRAM:8G...","[M.2(NVMe), /, 500GB, /, 저장 슬롯, :, 2개, /]","[무선랜: 802.11ax(Wi-Fi 6E) / 유선랜:, 기가비트, /]","[HDMI 2.0, /, mini DP, /, 웹캠(HD), /]","[썬더볼트4, :, 1개(USB-C겸용), / USB-C: 1개 / USB-A: 2...","[DP Alt Mode, /]","[RGB 라이트, /, ㅗ형 방향키, /, 숫자 키패드(4열), /]","[배터리, :, 73Wh, /, 어댑터, :, 280W, /, 충전단자, :, DC...","[두께, :, 27.8mm, /, 무게, :, 2.8kg, / 쿨링팬: 2개 / 색..."
7,에이서 스위프트 GO 16 OLED SFG16-71-51BY,"[1,188,980]",노트북,미포함(프리도스),"[40.8cm(16인치), /, 3200x2000(WQHD+), /, 500nit,...","[인텔, /, 코어i5-13세대, /, i5-13420H (2.1GHz), /, 옥...","[LPDDR5(온보드), /, 램 용량, :, 16GB, / 램 교체: 불가능 /]",[내장그래픽 / UHD Graphics /],"[M.2(NVMe), /, 512GB, /, 저장 슬롯, :, 2개, /]",[무선랜: 802.11ax(Wi-Fi 6E) /],"[HDMI 2.1, / 웹캠(QHD) /]","[썬더볼트4, :, 2개(USB-C겸용), / USB-A: 2개 / MicroSD카...",[지문인식(전원버튼) /],"[키보드 라이트, /, ㅡ형 방향키, /, 숫자 키패드(3열), /]","[배터리, :, 50Wh, /, 어댑

In [ ]:
merged_df.to_csv('output.csv', index=True, encoding='utf-8-sig')